# **Base line** Protein homo

## Required libraries and configuration

In [1]:
#!pip install -q flwr
!pip install -q flwr[simulation]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 33.2 MB/s eta 0:00:00



Import required libraries

In [2]:
import os

import re
import time
import random
import sys

from typing import Dict, Optional, Tuple, List
from collections import OrderedDict

import flwr as fl
from flwr.common import Metrics

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras import models, layers, losses, metrics, optimizers

from imblearn.datasets import fetch_datasets

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics as skmetrics
from sklearn.metrics import confusion_matrix

Define some parameters for the simulation, such as the number of clients in the federated scenario, the number of federated rounds, the number of epochs of each client before communicating, and the batch size for training phase

In [4]:
# Some parameters
NUM_CLIENTS = 10 # Number of clients in the federated scenario
NUM_ROUNDS = 10 #5 Number of learning rounds in the federated computation
NUM_EPOCHS = 5 #3 Number of epochs that the local dataset is seen each round
BATCH_SIZE = 20 # Batch size for training phase

# Define the seed for random numbers
seed = 10
np.random.seed(seed)
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)
random.seed(seed)

## Loading and preparing the input data

The Sentiment140 dataset is not available in TFF, so it needs to be loaded from other source such as the tfds (tensorflow datasets) library. Then, it is adapted to the TFF format, so it can be used to train a model using TFF.

Note: We download the full Sentiment140 dataset, but in order to be able to execute the experiments in reasonable time, we are going to use a portion of it. In this notebook we are selecting just 1% of the data for training and 10% for testing purposes.

In [5]:
dset_name = 'protein_homo'
dset = fetch_datasets()[dset_name]

In [6]:
X = pd.DataFrame(StandardScaler().fit_transform(dset.data))
y = dset.target

y[y == -1] = 0
X['label'] = y

dtrain, dtest = train_test_split(X, test_size=0.2, random_state=42, stratify=y)

In [11]:
dtest = dtest.iloc[:-1,:]

In [7]:
dtrain.shape

(116600, 75)

In [8]:
dtrain['label'].value_counts()

0    115563
1      1037
Name: label, dtype: int64

Select the text and polarity columns from the original data, and transform it to a dataframe so it can be later used in TFF.

At this point, we do not select the 'user' column because we will create random IID partitions, so the 'user' column is not neccesary. If the user want to try it with non-IID partition, he/she may also keep the 'user' column and use it as client identifier instead of the following random user ID.

In [9]:
# Create a random list of ids. Each instance is given a random id, indicating the client where will be distributed
ids_train = [i for i in range(NUM_CLIENTS) for _ in range(len(dtrain)//NUM_CLIENTS)]
random.Random(seed).shuffle(ids_train)

# Add the id assignment to the dataframe
dtrain['user'] = ids_train
unique_ids_train = list(set(ids_train))

In [14]:
# Do the same with the test data
ids_test = [i for i in range(NUM_CLIENTS) for _ in range(len(dtest)//NUM_CLIENTS)]
random.Random(seed+1).shuffle(ids_test)
dtest['user'] = ids_test
unique_ids_test = list(set(ids_test))

<ipython-input-14-698a78786656>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['user'] = ids_test


For the sake of simplicity, in this notebook we will be dealing with a binary problem. For that purpose, we remove the neutral tweets, so the classifier's aim is to differentiate between positive and negative tweets.

Create the DataLoaders for each client. It creates an array of datasets, one for each client.

In [15]:
# Create DataLoaders for each client
train_data = []
test_data = []
for i in range(NUM_CLIENTS):
    train_data.append(dtrain.loc[dtrain['user']==unique_ids_train[i]])
    test_data.append(dtest.loc[dtest['user']==unique_ids_test[i]])

## Create a Deep Learning model

In this case we use a model composed by a pre-trained model from tfhub, as well as dense layers. The pre-trained model is not updated in the example; however, the ``trainable`` parameter can be set to ``True``, so such layers are also fine-tuned in the collaborative training.

Note that any network architecture supported by keras can be used. Besides, Flower also supports PyTorch models, but in contrast to Use Case 1, we use a TFF model here to provide a variety of options.

In [16]:
def create_keras_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=[74]))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    # Compile the model
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.AUC(from_logits=True)]
    )

    return model

## Training in the federated scenario

First, we create a FlowerClient class, that includes the information of each simulated client. The class has three methods:
 * `get_parameters`: Get the parameters of the model to send them to the server
 * `fit`: Reveives the model parameters from the server, trains it with local data, and return the updated model parameters to the server
 * `evaluate`: Receives the model from the server and evaluates it with local data

In [17]:
# Class-independent methods to get and set the parameters of a model
def get_parameters(model):
        return model.get_weights()

def set_parameters(model, parameters):
    model.set_weights(parameters)

In [18]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, client_train_data, client_test_data):
        # Init the client setting the x (text) and the y (polarity) for both train and testing
        self.model = model

        client_train_data = client_train_data.drop('user', axis=1)
        client_test_data = client_test_data.drop('user', axis=1)

        x_train, y_train = client_train_data.drop('label',axis=1), client_train_data['label'].to_numpy()
        x_test, y_test = client_test_data.drop('label',axis=1), client_test_data['label'].to_numpy()

        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

    def get_properties(self, config):
        """Get properties of client."""
        raise Exception("Not implemented")

    def get_parameters(self, config):
        """Get parameters of the local model."""
        return get_parameters(self.model)

    def fit(self, parameters, config):
        """Train parameters on the locally held training set."""
        # Update local model parameters
        set_parameters(self.model, parameters)

        # Get hyperparameters for this round
        batch_size: int = BATCH_SIZE
        epochs: int = NUM_EPOCHS

        # Train the model using hyperparameters from config
        history = self.model.fit(
            self.x_train,
            self.y_train,
            batch_size,
            epochs,
            verbose=2
        )

        # Return updated model parameters and results
        parameters_prime = get_parameters(model=self.model)
        num_examples_train = len(self.x_train)
        results = {
            "loss": history.history["loss"][0],
            #"accuracy": history.history["accuracy"][0],

        }
        return parameters_prime, num_examples_train, results

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""

        # Update local model with global parameters
        set_parameters(self.model, parameters)

        # Get config values
        steps: int = 10

        # Evaluate global model parameters on the local test data and return results
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, 1)
        num_examples_test = len(self.x_test)

        y_pred = self.model.predict(self.x_test)
        y_pred = [1 if pred >= 0.5 else 0 for pred in y_pred]

        cm = confusion_matrix(self.y_test, y_pred)
        tn = cm[0][0]
        fn = cm[1][0]
        tp = cm[1][1]
        fp = cm[0][1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)

        cr = skmetrics.classification_report(self.y_test, y_pred)
        print(cr)

        return loss, num_examples_test, {"TPR*TNR": tpr*tnr}

To simulate the federated scenario in a single machine, the client_fn method allows to create FlowerClients on demand, given the client id.

Note that each client is passed both training and testing local data, so the evaluation over test data is done during the simulation itself.

In [19]:
def client_fn(cid: str) -> FlowerClient:
    # Load model
    model = create_keras_model()

    # Note: each client gets a different train/test data
    client_train_data = train_data[int(cid)]
    client_test_data = test_data[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(model, client_train_data, client_test_data)

In order to show averaged evaluations metrics beyond loss, we should define a method to do that; in this case, the accuracy is weighted averaged.

In [20]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["TPR*TNR"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"TPR*TNR": sum(accuracies) / sum(examples)}

Define a training strategy with the weighted FedAvg algorithm.

Then, start the simulation indicating the method to create clients, the number of clients in the simulation, the number of rounds, and the strategy (i.e., the FedAvg strategy to combine local updates). The simulation covers both the federated model training as well as evaluating the model with each local test data.

In [21]:
# Define the configuration during training/fitting the model. It is used later when defining the FedAvg strategy.
def fit_config(server_round: int):
    config = {
        "batch_size": BATCH_SIZE,
        "local_epochs": NUM_EPOCHS
    }
    return config

# Define the configuration during evaluation. It is used later when defining the FedAvg strategy.
def evaluate_config(server_round: int):
    return {"val_steps": 5}

In [22]:
# Define strategy
strategy = fl.server.strategy.FedAvg(
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl_sim = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-07-20 11:38:33,591 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-07-20 11:38:36,486	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-20 11:38:38,462 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3941437440.0, 'memory': 7882874880.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3941437440.0, 'memory': 7882874880.0}
INFO flwr 2023-07-20 11:38:38,473 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-07-20 11:38:38,484 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=1874) 2023-07-20 11:38:40.620495: W tensorflow/compiler

(launch_and_fit pid=1874) Epoch 1/5


(pid=1873) 2023-07-20 11:38:49.292500: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_fit pid=1874) 583/583 - 4s - loss: 0.0239 - auc_1: 0.9150 - 4s/epoch - 7ms/step
(launch_and_fit pid=1874) Epoch 2/5
(launch_and_fit pid=1874) 583/583 - 2s - loss: 0.0103 - auc_1: 0.9659 - 2s/epoch - 3ms/step
(launch_and_fit pid=1874) Epoch 3/5
(launch_and_fit pid=1873) Epoch 1/5
(launch_and_fit pid=1874) 583/583 - 2s - loss: 0.0071 - auc_1: 0.9880 - 2s/epoch - 3ms/step
(launch_and_fit pid=1874) 583/583 - 2s - loss: 0.0036 - auc_1: 0.9946 - 2s/epoch - 3ms/step
(launch_and_fit pid=1873) 583/583 - 4s - loss: 0.0283 - auc: 0.9182 - 4s/epoch - 7ms/step
(launch_and_fit pid=1874) 583/583 - 3s - loss: 0.0024 - auc_1: 0.9999 - 3s/epoch - 4ms/step
(launch_and_fit pid=1874) Epoch 1/5 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(launch_and_fit pid=1873) 583/583 - 3s - loss: 0.0085 - auc: 0.9929 - 3s/epoch - 6

DEBUG flwr 2023-07-20 11:40:07,682 | server.py:232 | fit_round 1 received 10 results and 0 failures
DEBUG:flwr:fit_round 1 received 10 results and 0 failures
WARNING flwr 2023-07-20 11:40:07,722 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-07-20 11:40:07,734 | server.py:168 | evaluate_round 1: strategy sampled 10 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 10 clients (out of 10)


...

DEBUG flwr 2023-07-20 11:58:40,166 | server.py:182 | evaluate_round 9 received 10 results and 0 failures
DEBUG:flwr:evaluate_round 9 received 10 results and 0 failures
DEBUG flwr 2023-07-20 11:58:40,171 | server.py:218 | fit_round 10: strategy sampled 10 clients (out of 10)
DEBUG:flwr:fit_round 10: strategy sampled 10 clients (out of 10)


92/92 [==============================] - 0s 1ms/step
(launch_and_evaluate pid=1873) 
(launch_and_fit pid=1874) Epoch 1/5
1532/2915 [==============>...............] - ETA: 6s - loss: 0.0107 - auc_89: 0.9486 [repeated 2x across cluster]
(launch_and_evaluate pid=1873) Epoch 1/5
1644/2915 [===============>..............] - ETA: 5s - loss: 0.0100 - auc_89: 0.9532 [repeated 2x across cluster]
(launch_and_fit pid=1874) 583/583 - 3s - loss: 0.0074 - auc_91: 0.9806 - 3s/epoch - 6ms/step
1915/2915 [==================>...........] - ETA: 4s - loss: 0.0103 - auc_89: 0.9630 [repeated 2x across cluster]
(launch_and_evaluate pid=1873) Epoch 2/5
(launch_and_evaluate pid=1873) Epoch 2/5
2406/2915 [=======================>......] - ETA: 1s - loss: 0.0095 - auc_89: 0.9727 [repeated 2x across cluster]
(launch_and_evaluate pid=1873) Epoch 2/5
(launch_and_evaluate pid=1873) Epoch 2/5
2703/2915 [==========================>...] - ETA: 0s - loss: 0.0119 - auc_89: 0.9512 [repeated 2x across cluster]
(launch_and

DEBUG flwr 2023-07-20 11:59:52,779 | server.py:232 | fit_round 10 received 10 results and 0 failures
DEBUG:flwr:fit_round 10 received 10 results and 0 failures
DEBUG flwr 2023-07-20 11:59:52,808 | server.py:168 | evaluate_round 10: strategy sampled 10 clients (out of 10)
DEBUG:flwr:evaluate_round 10: strategy sampled 10 clients (out of 10)


 450/2915 [===>..........................] - ETA: 12s - loss: 0.0299 - auc_96: 0.9369
(launch_and_fit pid=1874) 583/583 - 1s - loss: 0.0020 - auc_95: 0.9999 - 1s/epoch - 3ms/step [repeated 2x across cluster]
92/92 [==============================] - 0s 2ms/step
(launch_and_evaluate pid=1874)               precision    recall  f1-score   support
(launch_and_evaluate pid=1874) 
(launch_and_evaluate pid=1874)            0       1.00      1.00      1.00      2887
(launch_and_evaluate pid=1874)            1       0.95      0.64      0.77        28
(launch_and_evaluate pid=1874) 
(launch_and_evaluate pid=1874)     accuracy                           1.00      2915
(launch_and_evaluate pid=1874)    macro avg       0.97      0.82      0.88      2915
(launch_and_evaluate pid=1874) weighted avg       1.00      1.00      1.00      2915
(launch_and_evaluate pid=1874) 
1710/2915 [================>.............] - ETA: 3s - loss: 0.0112 - auc_97: 0.9472
(launch_and_evaluate pid=1873)  [repeated 5x acr

DEBUG flwr 2023-07-20 12:00:50,857 | server.py:182 | evaluate_round 10 received 10 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 10 results and 0 failures
INFO flwr 2023-07-20 12:00:50,864 | server.py:147 | FL finished in 1326.503860028
INFO:flwr:FL finished in 1326.503860028
INFO flwr 2023-07-20 12:00:50,872 | app.py:218 | app_fit: losses_distributed [(1, 0.01258629965595901), (2, 0.014242459135130048), (3, 0.014687007060274482), (4, 0.01451080576516688), (5, 0.014248480182141065), (6, 0.015549019537866115), (7, 0.015012042410671712), (8, 0.016091710375621914), (9, 0.017704538628458978), (10, 0.01816059756092727)]
INFO:flwr:app_fit: losses_distributed [(1, 0.01258629965595901), (2, 0.014242459135130048), (3, 0.014687007060274482), (4, 0.01451080576516688), (5, 0.014248480182141065), (6, 0.015549019537866115), (7, 0.015012042410671712), (8, 0.016091710375621914), (9, 0.017704538628458978), (10, 0.01816059756092727)]
INFO flwr 2023-07-20 12:00:50,875 | app.py:219 | app_fi

## Evaluation with test data

The evaluation has been done during the simulation. Following, we show the averaged results over test data.
The result of the simulation includes the results on all rounds, so we retrieve those of the last round.

In [23]:
print('Test data, \t Loss={:.4f}, \t Accuracy={:.4f}'.format(fl_sim.losses_distributed[-1][1], fl_sim.metrics_distributed['TPR*TNR'][-1][1]))

Test data, 	 Loss=0.0182, 	 Accuracy=0.8277


In [24]:
fl_sim

History (loss, distributed):
	round 1: 0.01258629965595901
	round 2: 0.014242459135130048
	round 3: 0.014687007060274482
	round 4: 0.01451080576516688
	round 5: 0.014248480182141065
	round 6: 0.015549019537866115
	round 7: 0.015012042410671712
	round 8: 0.016091710375621914
	round 9: 0.017704538628458978
	round 10: 0.01816059756092727
History (metrics, distributed, evaluate):
{'TPR*TNR': [(1, 0.7904122227425018), (2, 0.8320417612931181), (3, 0.8316080825817789), (4, 0.8359620415033249), (5, 0.8382993700082305), (6, 0.8501750554067845), (7, 0.8385282763598496), (8, 0.8229648488455851), (9, 0.8419395250928251), (10, 0.8276837238168127)]}